In [1]:
%pip install mosestokenizer

  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=b0808308a08a48928678befeea974263228dc26efa530828b33f6894d11a5357
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=8ecc40ec71996e4f12f622f8374246460ef55bb405d8adbdafd10b6ab850efee
  Stored in directory: /root/.cache/pip/wheels/e1/af/b1/99b57a06dda78fdcee86d2e22c64743f3b8df8f31cfc04baf7
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6147 sha256=772a87bacb734944265c90655ab27d36d276c8c3582455303075cf5c0cb9618d
  Stored in directory: /root/.cache/pip/wheels/05/ee/10/33257b0801ac6a912c841939032c16da1eb3db377afe1443e5
Successfully built mosestokenizer toolwrapper uctools
Note: you may 

In [2]:
!pwd

/kaggle/working


In [3]:
HTML = "https://www.gutenberg.org/cache/epub/2814/pg2814-images.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
dubliners = requests.get(HTML)

In [6]:
assert dubliners.status_code == 200

In [7]:
soup = BeautifulSoup(dubliners.text, 'html.parser')

In [8]:
body = soup.find("body")

In [9]:
text = []
for chapter in body.findAll("div", {"class": "chapter"}):
    for element in chapter.findChildren():
        if element.name == "section" and element.get("id") in ["pg-header", "pg-footer"]:
            continue
        stripped = element.text.strip()
        if stripped != "":
            text.append(stripped)

In [10]:
from mosestokenizer import MosesSentenceSplitter

In [11]:
sents = []
try:
    with MosesSentenceSplitter('en') as splitsents:
        for para in text:
          if para == "":
              continue
          sents += splitsents([para.replace("\r\n", " ")])
except Exception as ex:
    print(ex, para)

In [12]:
import re
def cleaner(text):
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("’ ", " ")
    text = text.replace("’", "'")
    text = text.replace("‘", " ")
    text = text.replace("\t", " ")
    text = text.replace("!...", " ")
    text = text.replace("....", " ")
    text = text.replace("...", " ")
    text = text.replace(":", " ")
    text = text.replace(";", " ")
    text = text.replace("!", " ")
    text = text.replace(",", " ")
    text = text.replace("?", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("—", " ")
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace("Rev. James", "Reverend James")
    text = text.replace("S. Catherine's", "Saint Catherine's")
    text = text.replace(".", " ")
    text = text.replace("&", " and ")
    text = text.replace(" 57E ", " fifty seven e ")
    text = text.replace(" 1st ", " first ")
    text = text.replace(" 6th ", " sixth ")
    text = text.replace(" 1895 ", " eighteen ninety five ")
    text = text.replace(" 1891", " eighteen ninety one")
    text = text.replace("1891", " eighteen ninety one")
    text = text.replace("65 ", " sixty five ")
    if text[-1] == ".":
        text = text[:-1]
    if text.endswith("'"):
        text = text[:-1]
    text = re.sub("  +", " ", text)
    return text.lower().strip()

In [13]:
clean = [cleaner(x) for x in sents]

In [14]:
with open("dubliners-clean.txt", "w") as outf:
    for line in clean:
        outf.write(line + "\n")

In [15]:
!apt-get update
!apt install -y build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev libboost-program-options-dev libboost-all-dev libeigen3-dev

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 https://packages.cloud.google.com/apt cloud-sdk InRelease [1620 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:6 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [55.8 kB]
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1905 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1308 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:10 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [4396 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4920 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4432 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/restr

In [16]:
%cd /tmp

/tmp


In [17]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 14185 (delta 117), reused 81 (delta 81), pack-reused 14009 (from 2)
Receiving objects: 100% (14185/14185), 5.91 MiB | 15.18 MiB/s, done.
Resolving deltas: 100% (8059/8059), done.


In [18]:
%cd kenlm

/tmp/kenlm


In [19]:
!mkdir build
%cd build
!cmake ..
!make -j 4

/tmp/kenlm/build
cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Fou

In [20]:
%cd /kaggle/working

/kaggle/working


In [21]:
!/tmp/kenlm/build/bin/lmplz -o 5 --text dubliners-clean.txt --arpa dubliners.arpa --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /kaggle/working/dubliners-clean.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 68157 types 7521
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:90252 2:2627305984 3:4926198784 4:7881917952 5:11494464512
Statistics:
1 7521 D1=0.613953 D2=1.27324 D3+=1.35029
2 36752 D1=0.801814 D2=1.15571 D3+=1.52297
3 56938 D1=0.904306 D2=1.32517 D3+=1.56145
4 60053 D1=0.964599 D2=1.51904 D3+=1.7929
5 56894 D1=0.977545 D2=1.72265 D3+=1.46157
Memory estimate for binary LM:
type      kB
probing 4794 assuming -p 1.5
probing 5724 assuming -r models -p 1.5
trie    2208 without quantization
trie    1173 assuming -q 8 -b 8 quantization 
trie    2041 assuming -a 22 array pointer compression
trie    1006 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
===

In [22]:
!cat dubliners-clean.txt | tr ' ' '\n'|sort|uniq > unigrams.txt

In [23]:
!/tmp/kenlm/build/bin/build_binary trie -a 255 -q 8 dubliners.arpa dubliners.klm

Reading dubliners.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Identifying n-grams omitted by SRI
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Quantizing
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Writing trie
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS
